In [22]:
import os
import sys
import torch
from glob import glob
from os.path import join, isfile, isdir, splitext, basename
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import  KFold

from ultralytics import YOLO
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

In [23]:
label_map = {"CFCBK": 0, "FCBK": 1, "Zigzag": 2}

In [24]:
name="wb_small_airshed"
tasks=['obb_labels','aa_labels']
basepath="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data"
satellite_type="sentinel"

In [25]:




file_names = []
for task in tasks:
    data_dir = join(basepath, satellite_type, f"{name}", f"{task}")
    file_names.append(
        np.array(
            list(map(lambda x: basename(x), sorted(glob(join(data_dir, "*.txt")))))
        )
    )

first_task_files = file_names[0]
for files in file_names:
    assert np.all(first_task_files == files), "Files are not same across tasks"

final_len = len(first_task_files)

kf = KFold(n_splits=4, random_state=0, shuffle=True)

for fold_id, (train_idx, test_idx) in enumerate(kf.split(np.arange(final_len))):
    fold_id = str(fold_id)
    train_files = first_task_files[train_idx]
    test_files = first_task_files[test_idx]

    for task in tasks:
        print("Fold", fold_id, "Task", task)
        label_dir = join(basepath, satellite_type, f"{name}", task)
        image_dir = join(basepath, satellite_type, f"{name}", "rgb")  # Assuming images live here
        print("Label dir", label_dir)
        print("Image dir", image_dir)
        save_dir = join(basepath, "crossval", f"{name}_{task}_{satellite_type}")

        if fold_id == "0":
            os.system(f"rm -rf {save_dir}")
            print("Removed existing folder:", save_dir)

        os.makedirs(join(save_dir, fold_id, "train", "images"), exist_ok=True)
        os.makedirs(join(save_dir, fold_id, "train", "labels"), exist_ok=True)
        os.makedirs(join(save_dir, fold_id, "test", "images"), exist_ok=True)
        os.makedirs(join(save_dir, fold_id, "test", "labels"), exist_ok=True)

        # Symlinking function
        def symlink_files(file_list, split_type):
            for fname in file_list:
                base = fname.replace(".txt", "")
                label_src = join(label_dir, fname)
                image_src = join(image_dir, base + ".png")

                label_dst = join(save_dir, fold_id, split_type, "labels", fname)
                image_dst = join(save_dir, fold_id, split_type, "images", base + ".png")

                if os.path.exists(label_src) and os.path.exists(image_src):
                    os.symlink(label_src, label_dst)
                    os.symlink(image_src, image_dst)
                else:
                    print(f"[WARN] Missing for {fname}: {label_src}, {image_src}")

        symlink_files(train_files, "train")
        symlink_files(test_files, "test")

        # Validation print
        train_path = join(save_dir, fold_id, "train")
        test_path = join(save_dir, fold_id, "test")

            # write data.yml
        with open(join(save_dir, fold_id, "data.yml"), "w") as f:
            f.write(f"train: {train_path}\n")
            f.write(f"val: {test_path}\n")
            f.write("nc: 3\n")
            f.write(f"names: {list(label_map)}\n")

    print("Train images", len(glob(join(train_path, "images", "*.png"))))
    print("Train labels", len(glob(join(train_path, "labels", "*.txt"))))
    print("Test images", len(glob(join(test_path, "images", "*.png"))))
    print("Test labels", len(glob(join(test_path, "labels", "*.txt"))))


Fold 0 Task obb_labels
Label dir /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/wb_small_airshed/obb_labels
Image dir /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/wb_small_airshed/rgb
Removed existing folder: /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/crossval/wb_small_airshed_obb_labels_sentinel
Fold 0 Task aa_labels
Label dir /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/wb_small_airshed/aa_labels
Image dir /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/wb_small_airshed/rgb
Removed existing folder: /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/crossval/wb_small_airshed_aa_labels_sentinel
Train images 93
Train labels 93
Test images 31
Test labels 31
Fold 1 Task obb_labels
Label dir /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/

In [26]:
from PIL import Image

# Path to your image
img_path = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/delhi_airshed/rgb/28.2056_76.8558.png"

# Open and check mode
with Image.open(img_path) as img:
    print(f"Image mode: {img.mode}")  # Will print 'RGB' or 'RGBA'


Image mode: RGB
